# Improve your machine learning training routine with cloud-based, containerized workflows 
**Carsten Frommhold, Paul Elvers**

## Introduction
Training machine learning models on a local machine in a notebook is a common task among data scientists.  It is the easiest way to get started, experiment and build a first working model. But for most businesses this is not a satisfying option: Today, making the most of your training data usually means to scale to gigabytes or petabytes of data, which do not easily fit into your local machine. Data is your most valuable asset and you would not want to use only small fraction of it due to technical reasons. Another problem that might occur when training a model on your local machine is putting it to use. A model that can only be used on your laptop is pretty useless. It should be available to a large group of consumers, deployed as a REST API or making batch predictions in a large data pipeline. If your model “works on your machine”, how does it get to production?

Another pitfall lies in the consistency between code used to train a model and the serialized model itself. By design, a notebook is well suited for exploratory analysis and development via trial and error. For example, it is possible to jump between individual cells. It is not enforced to keep a given top-down order. This makes it difficult to check whether the code exactly matches the model fit. Fitting a model in a notebook, persisting it, change the notebook afterwards and do a git commit is not a desired workflow and might lead to non-traceable issues. In an optimal workflow, models and associated code should be kept consistent with each other in the sense that it should be possible to fit the model in a new environment again.

We demonstrate how many of the issues can be circumvented, by moving your training routine to the cloud. We use the AWS Service Sagemaker to execute a containerised training routine that will allow us to a) scale our training job to whatever sizes of training data, b) support our routines with single or multiple GPUs if needed, and c) easily deploy our model after the training has succeeded. The training and deployment routine will be written in a python script, which can easily be version controlled, and allows us to link code and model. The script is passed via the Sagemaker Python SDK to a container, running a docker image with all dependencies needed for the job. The main benefit of using Sagemaker is that instead of having to manage the infrastructure running the container, all the infrastructure management is done for you by AWS. All that is required is to make a few calls with the Python Sagemaker SDK and everything else is taken care of.

## Prerequisites 
To try this demo out by yourself, you will need access to an AWS account. The easiest way would be to use an AWS Sagemaker notebook instance (you can find more information about how to configure a notebook instance [here](https://docs.aws.amazon.com/sagemaker/latest/dg/howitworks-create-ws.html)), but the code can be run in any compute environment, given that authentication and permissions are provided.

## Data

In this example we will be using the Palmer Penguin Dataset, which provides a suitable alternative to the frequently used Iris dataset. It contains information about various penguins. You can read more about it [here](https://allisonhorst.github.io/palmerpenguins/articles/intro.html). The objective we will be solving with our machine learning algorithm is predicting the sex of a penguin (male/female) by using various attributes of the penguin (e.g. flipper length, bill length, species, island) as our features.
![Penguins](https://camo.githubusercontent.com/4ff5cb2b783a76207da2b80b5b5e97b045f39f05cf76e8a21a34a18dc5f492cd/68747470733a2f2f616c6c69736f6e686f7273742e6769746875622e696f2f70616c6d657270656e6775696e732f6d616e2f666967757265732f6c7465725f70656e6775696e732e706e67)

In [ ]:
!pip install palmerpenguins

In [ ]:
from palmerpenguins import load_penguins

In [ ]:
penguins = load_penguins()
penguins.head(3)

Minimal data procesing is required: We drop entries with null values as well as duplicates. Then, we perform a train-test-split and save the data on our working directory as well as in AWS s3 storage.

We use the library dotenv to load the following environment variables. They will be needed for executing the job further below and contain information on the role used for execution as well as the paths to the data and the model storage. 

In [ ]:
import numpy as np
import os

In [ ]:
%load_ext dotenv
%dotenv

sagemaker_role= os.getenv("SAGEMAKER_ROLE")
s3_data_storage_path=os.getenv("DATA_PATH")
s3_output_storage_path=os.getenv("OUTPUT_PATH")

In [ ]:
penguins.dropna(inplace=True)
penguins.drop_duplicates(inplace=True)

features = [
    "bill_length_mm",
    "bill_depth_mm",
    "flipper_length_mm",
    "species",
    "island",
]

target = "sex"

test_amount = 0.3
train = [np.random.uniform() >= test_amount for _ in range(len(penguins))]
test = [not train_flag for train_flag in train]

X_train = penguins[train][features]
y_train = penguins[train][target]
X_test = penguins[test][features]
y_test = penguins[test][target]

try:
    os.mkdir("data/")
    print("Created data/ directory.")
except FileExistsError:
    print("Data directory already exists")
    

for raw_data_bucket in ["data/", s3_data_storage_path]:

    X_train.to_csv(os.path.join(raw_data_bucket, "X_train.csv"), index=False)
    y_train.to_csv(os.path.join(raw_data_bucket, "y_train.csv"), index=False)
    X_test.to_csv(os.path.join(raw_data_bucket, "X_test.csv"), index=False)
    y_test.to_csv(os.path.join(raw_data_bucket, "y_test.csv"), index=False)
    print(f"Stored data in '{raw_data_bucket}'.")

## Model Training


To execute the training and deployment routine, we need to write a python script. The crucial part for the training lies in the *__main__* clause. It reads the data, instantiates a pipeline and trains the the model. Here, a minimal preprocessing of one-hot-encoding and standard scaling is chosen. LogisticRegression acts as a baseline model. The model is then serialized and saved given the model directory. 

The script takes four arguments. First, we need to define input path for the training data. It assumes the existence of two files: X_train.csv and y_train.csv. The differentiation between categorical and numerical variables is explicitly given in these example. Finally, the output path for the serialized model is defined. Using these arguments makes it convenient to run the training routine on our local machine and via SageMaker in the cloud.

In [ ]:
%%writefile train_and_deploy.py

import os
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
import joblib
import argparse
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import make_column_transformer
from sklearn.pipeline import Pipeline


def model_fn(model_dir):
    model = joblib.load(os.path.join(model_dir, "model.joblib"))
    return model

def float_if_number(entry):
    try:
        return float(entry)
    except ValueError:
        return entry

def input_fn(request_body, content_type):
    if content_type == 'text/csv':
        features = []
        for r in request_body.split('|'):
            mapped_row = list(map(float_if_number,r.split(',')))
            features.append(mapped_row)
            features_as_array = np.array(features)
        return features_as_array
    else:
        raise ValueError("Thie model only supports text/csv input")

def predict_fn(input_data, model):
    return model.predict(pd.DataFrame(input_data, columns=model.steps[0][1]._feature_names_in))

def output_fn(prediction, content_type):
    return str(prediction)


if __name__ == "__main__":
    
    parser = argparse.ArgumentParser()

    parser.add_argument('--train', type=str, default="/opt/ml/input/data/train")
    parser.add_argument('--num_features', type=str) 
    parser.add_argument('--cat_features', type=str)
    parser.add_argument('--model-dir', type=str, default="/opt/ml/model")
    args, _ = parser.parse_known_args()
    
    train_path = args.train
    num_features = args.num_features.split()
    cat_features = args.cat_features.split()
    model_dir = args.model_dir

    X_train = pd.read_csv(os.path.join(train_path, "X_train.csv"))
    y_train = pd.read_csv(os.path.join(train_path, "y_train.csv"))
    
    preprocessor = make_column_transformer(
        (StandardScaler(), num_features),
        (OneHotEncoder(sparse=False), cat_features),
    )
    
    model = LogisticRegression(class_weight="balanced", solver="lbfgs")
    
    pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                               ('model', model)])
    
    pipeline.fit(X_train, np.ravel(y_train))
    
    model_output_directory = os.path.join(model_dir, "model.joblib")
    print("Model saving path {}".format(model_output_directory))
    joblib.dump(pipeline, model_output_directory)

The script also contains several serving functions that Sagemaker requires for model serving via the Sagemaker model endpoint service. These functions comprise of model_fn() ensuring that the model gets loaded from file, input_fn() handling the input in a way that it can be used for calling the predict() function on the model, the predict_fn() which calls predict on the model and the output_fn(), which will convert the model output to a format that can be send back to the caller. 

For testing purposes the script is also callable on our local machine, or on the instance on which the notebook is running.

In [ ]:
!python3 train_and_deploy.py --train ./data  \
                             --num_features "bill_length_mm bill_depth_mm flipper_length_mm"  \
                             --cat_features "species island"  \
                             --model-dir ./  

In order to run the training routine in the cloud, we use the SKLearn object from the Pyhton SDK. It is the standard interface for scheduling and defining model training and deployment of scikit-learn models. We specify the resources needed, the framework version, the entry point, the role as well as the output_path which will be the model-dir argument. Further arguments like the numerical and categorical feature list can be passed via the hyperparameters dictionary. 

When calling fit(), Sagemaker will automatically launch a container with a scikit-learn image and execute the training script. The dictonary that we pass with a single keyword "train" to the fit() function specifies the path to the processed data in S3. The training data is copied from there into the training container. The SKLearn object will move the model artifacts to the desired output path in S3, defined via the keyword "output_path" in its definition. 

In [ ]:
from sagemaker import get_execution_role
from sagemaker.sklearn.estimator import SKLearn

In [ ]:
sklearn = SKLearn(
    entry_point="train_and_deploy.py",
    framework_version="0.23-1", 
    instance_type="ml.m5.xlarge", 
    role=sagemaker_role,
    hyperparameters={
        "num_features": "bill_length_mm bill_depth_mm flipper_length_mm",
        "cat_features": "species island"
    },
    output_path=s3_output_storage_path
)

In [ ]:
sklearn.fit({"train": s3_data_storage_path})

Instantiating the SKLearn object and calling the fit() method are everything that is needed to launch the training routine as a containerized workflow. We could easily scale the resources, for example by choosing a bigger instance or use GPU support for our training. We could also make our calls automatically using a workflow orchestration tool such as Airflow or AWS Step Function, or trigger our training each time we merge our training routine with a CI/CD pipeline.

## Model deployment

After evaluating our model, we can now go on and deploy it. To do so, we only have to call deploy() on the SKlearn object that we used for model training. A model endpoint is now booted in the background. 

In [ ]:
predictor = sklearn.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.xlarge")

We can test the endpoint by passing the top 10 rows of our test dataset to the endpoint. We will receive labeled predictions as to whether the penguin is a male or a female based on it's attributes. 

In [ ]:
import boto3

to_be_predicted = X_test.head(10).values.tolist()

request_body = ""
for sample in to_be_predicted:
    request_body += ",".join([str(n) for n in sample]) + "|"
request_body = request_body[:-1] 
print("*"*20)
print(f"Calling Sagemaker Endopint with the following request_body: {request_body}")

client = boto3.client('sagemaker-runtime')

endpoint_name = predictor.endpoint_name
content_type = 'text/csv'

response = client.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=request_body,
    ContentType=content_type
    )
response_from_endpoint = response['Body'].read().decode("utf-8")
print("*"*20)
print(f"Response from Endpoint: {response_from_endpoint}")

At the end of our journey, the endpoint should be shut down.

In [ ]:
predictor.delete_endpoint()

## Outlook
We demonstrated how a machine learning training routine can be moved to a cloud-based execution using a dedicated container in AWS. Compared to a local workflow it provides numerous advantages, such as scalability of compute ressources (one simply has to change parameters on the SKLearn object) and reproducibility of results: training scripts can easily be versioned and training jobs automatically be triggered, allowing to connect model training and version control. Most importantly, it reduces the barrier between development and production: Deploying a trained model only requires a single method call.  

Of course, the example provided here is not solving all technical hurdles of MLOps and more features are usually needed to build a mature ml-platform. Ideally, we would want to have a model registry, where we store and manage models and artifacts, an experimentation tracking platform to log all our efforts to improve the model, and perhaps also a data versioning tool. But moving your local training routine into the cloud is already a big step forward!

There are also many possibilities to extend the training routine and adapt it towards specific needs: If special python libraries or dependencies are used for the algorithm, a custom docker image can be pushed to AWS Elastic Container Registry (ECR) and then be used in the Sagemaker training routine. If data processing becomes more complex, one can use a processing container to decapsulate this step. Also, if the endpoint is used in production, it is advisable to develop a REST API on top of the deployed Sagemaker endpoint, allowing to better handle security constraints as well as logical heuristics and preprocessing of your API calls. And of course a model evaluation step that calculates metrics on the vaildation data set should be included, but we will dive deeper into model evalutation in one of our next blog posts. Stay tuned!